In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import re
import torch
import urllib.request
from torch.utils.data import Dataset, DataLoader
from transformers import PreTrainedTokenizerFast


###  챗봇 데이터 다운로드

In [2]:
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
    filename="ChatBotData.csv",
)
Chatbot_Data = pd.read_csv("ChatBotData.csv")

# Test 용으로 300개 데이터만 처리한다.
Chatbot_Data = Chatbot_Data[:300]
Chatbot_Data.head()


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


Tokenizer는 모델에 어떠한 입력을 넣어주기 위해서 전처리를 담당합니다. 토크나이저는 허깅페이스의 PreTrainedTokenizer 인 GPT2Tokenizer 를 사용합니다. Tokenizer들은 크게 3가지 기능을 제공합니다.

1. Tokenizing : 입력 문자열을 token id로 변환(encoding), token id를 다시 문자열로 변환(decoding)의 기능
2. 기존의 구조(BPE, Sentencepiece 등)에 독립적으로 추가적인 token들을 추가하는 기능
3. Special token들을 (mask, BOS, EOS 등) 관리하는 기능

In [3]:
Q_TKN = "<usr>"  # 질문 시작 토큰
A_TKN = "<res>"  # 답변 시작 토큰
SENT = "<sent>"  # 문장 구분 토큰
EOS = "</s>"  # 문장 종료 토큰
PAD = "<pad>"  # 패딩 토큰
MASK = "<unused0>"  # 마스크 토큰
BOS = "</s>"

사용된 파라메터의 의미는 다음과 같습니다.

* bos_token : 문장의 시작을 나타내는 token
* eos_token : 문장의 끝을 나타내는 token
* unk_token : 모르는 단어를 나타내는 token
* pad_token : 동일한 batch 내에서 입력의 크기를 동일하게 하기 위해서 사용해는 token
* PreTrainedTokenizer 에서 제공되는 함수는

* tokenize() : tokenizer를 이용해서 string을 token id의 리스트로 변환한다.
* get_added_vocab() : token to index에 해당하는 dict를 리턴한다.
* batch_decode() : token id로 구성된 입력을 하나의 연결된 string으로 출력한다.
* convert_ids_to_tokens() : token id 의 리스트를 token으로 변환한다.
* skip_special_tokens=True로 하면 decoding할 때 special token들을 제거한다.
* convert_tokens_to_ids() : token string의 리스트를 token id 또는 Token id의 리스트로 변환한다.
* decode() : tokenizer 와 vocabulary를 이용해서 token id를 string으로 변환한다. * skip_special_token=True로 지정하면 speical token들을 제외한다.
* encode() : token string을 token id 의 리스트로 변환한다.
* add_special_tokens=False로 지정하면 token id로 변환할 때 special token들을 제외한다. padding을 통해서 padding token을 어떻게 추가할지조 지정할 수 있다.

In [4]:
# 허깅페이스 transformers 에 등록된 사전 학습된 koGTP2 토크나이저를 가져온다.
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token=BOS, eos_token=EOS, unk_token="<unk>", pad_token=PAD, mask_token=MASK,)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


### 데이터 셋 정의

In [5]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len

        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)


### 배치 데이터를 만들기 위해 collate_batch 정의

In [6]:
def collate_batch(batch):
    data = np.array([item[0] for item in batch])
    mask = np.array([item[1] for item in batch])
    label = np.array([item[2] for item in batch])
    return (torch.LongTensor(data),
            torch.LongTensor(mask),
            torch.LongTensor(label))


### Dataset DataLoader 정의

In [7]:
train_set = ChatbotDataset(Chatbot_Data, max_len=40)

#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=2, shuffle=True, collate_fn=collate_batch,)

### 데이터 로더를 사용하여 테스터를 생성


In [8]:
print("start")
for batch_idx, samples in enumerate(train_dataloader):
    token_ids, mask, label = samples
    print("token_ids ====>", token_ids)
    print("mask ====>", mask)
    print("label ====>", label)

print("end")

start


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


token_ids ====> tensor([[    2, 31100, 10175,  ...,     3,     3,     3],
        [    2, 10411, 13243,  ...,     3,     3,     3],
        [    2, 17542, 16353,  ...,     3,     3,     3],
        ...,
        [    2, 19787, 42226,  ...,     3,     3,     3],
        [    2,  9120, 36598,  ...,     3,     3,     3],
        [    2,   739,  6910,  ...,     3,     3,     3]])
mask ====> tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
label ====> tensor([[9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        ...,
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3]])
token_ids ====> tensor([[    2, 10715, 27120,  ...,     3,     3,     3],
        [    2,  9716, 10056,  ...,     3,     3,     3],
        [    2,  9065,   394,  

### Dataloader

In [ ]:
!pip install pytorch_lightning==1.6.0


In [ ]:
!pip install transformers

### 라이브러리 불러오기

In [11]:
import numpy as np
import pandas as pd

import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.lightning import LightningModule

from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
from transformers import GPT2Tokenizer

import re


### 토크나이저에서 사용되는 스페셜 토큰들 정의

In [12]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

### 허깅페이스 transformers에 등록된 사전 학습 모델 KkGPT2 모델 가져온다.

In [13]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK)
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(device)

cuda:0


In [15]:
train_set = ChatbotDataset(Chatbot_Data, max_len=40)

# 윈도우 환경에서는 num_workers는 0 , 리눅스는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=2, shuffle=True, collate_fn=collate_batch)

### 모델을 추가로 학습하기 위해 .train() 사용

In [16]:
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

### 하이퍼 파라메터 선언

In [17]:
learning_rate = 3e-4
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 10
Sneg = -1e18

### 학습

In [18]:
print("Device of token_ids:", token_ids.device)
print("Device of mask:", mask.device)
print("Device of label:", label.device)
print("Device of model:", next(model.parameters()).device)


Device of token_ids: cpu
Device of mask: cpu
Device of label: cpu
Device of model: cuda:0


In [24]:
token_ids = token_ids.to(model.device)
print("Device of token_ids:", token_ids.device)
print("Device of mask:", mask.device)
print("Device of label:", label.device)

Device of token_ids: cuda:0
Device of mask: cpu
Device of label: cpu


In [27]:
print ("start")
for epoch in range(epochs):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples

        token_ids = token_ids.to(model.device)
        mask = mask.to(model.device)
        label = label.to(model.device)
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print ("end")

start
end


In [32]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + SENT + A_TKN + a)).unsqueeze(dim=0)
            input_ids = input_ids.to(model.device)
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))


user > 카페갈래
Chatbot > 청소를 좋아하시나봐요
user > quit


## KoGpt Tset

In [ ]:
import torch
from transformers import GPT2LMHeadModel
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')
tokenizer.tokenize
("안녕하세요. 한국어 GPT-2 입니다.😤:) 답변해주세요.")


In [ ]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

text = 'kpop 하면 떠오르는 그룹 '
input_ids = tokenizer.encode(text)
gen_ids = model.generate(torch.tensor([input_ids]),
                         max_length=128,
                         repetition_penalty=2.0,
                         pad_token_id = tokenizer.pad_token_id,
                         eos_token_id = tokenizer.eos_token_id,
                         bos_token_id = tokenizer.bos_token_id,
                         use_cache=True)
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)
